<a href="https://colab.research.google.com/github/burrows99/NLP_Project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Data Loading

In [1]:
%pip install datasets transformers sklearn-crfsuite seqeval pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=41e9d6377d5faef8a583cdfa29cf46574d4fb990da6a64aeffb1fe789a7be570
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
  

In [2]:
import numpy as np
import torch
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import pos_tag
from nltk.util import ngrams
from collections import defaultdict
from datasets import load_dataset
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite import CRF, metrics
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from torch import nn
from seqeval.metrics import classification_report


# Load dataset and describe it

In [3]:
dataset = load_dataset("surrey-nlp/PLOD-CW-25")
splits = list(dataset.keys())

for split in splits:
    print(f"\n{split.upper()} Split:")
    print("="*50)
    df = dataset[split].to_pandas()
    print(df.head())
    print("="*50)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/268 [00:00<?, ?B/s]

PLOD-CW-25-Train.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

PLOD-CW-25-Test.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

PLOD-CW-25-Val.parquet:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/150 [00:00<?, ? examples/s]


TRAIN Split:
                                              tokens  \
0  [MRR, ,, mortality, rate, ratio, ;, TBI, ,, tr...   
1  [HPC, ,, hippocampus, ;, mOFC, ,, medial, OFC,...   
2  [In, addition, to, the, effects, on, cell, gro...   
3  [Abbreviations, :, HC, ,, C, -, terminal, rece...   
4   [LL, :, lower, limit, ,, UL, :, upper, limit, .]   

                                            pos_tags  \
0  [NOUN, PUNCT, NOUN, NOUN, NOUN, PUNCT, PROPN, ...   
1  [PROPN, PUNCT, NOUN, PUNCT, NOUN, PUNCT, ADJ, ...   
2  [ADP, NOUN, ADP, DET, NOUN, ADP, NOUN, NOUN, C...   
3  [NOUN, PUNCT, PROPN, PUNCT, NOUN, PUNCT, ADJ, ...   
4  [PROPN, PUNCT, ADJ, NOUN, PUNCT, PROPN, PUNCT,...   

                                            ner_tags  
0  [B-AC, O, B-LF, I-LF, I-LF, O, B-AC, O, B-LF, ...  
1  [B-AC, O, B-LF, O, O, O, O, B-AC, O, B-AC, O, ...  
2  [O, O, O, O, O, O, O, O, O, O, O, O, B-AC, B-A...  
3  [O, O, B-AC, O, B-LF, I-LF, I-LF, I-LF, I-LF, ...  
4   [B-AC, O, B-LF, I-LF, O, B-AC, O,

# Split dataset

In [4]:
train_tokens = dataset['train']['tokens']
train_tags = dataset['train']['ner_tags']
train_pos = dataset['train']['pos_tags']

val_tokens = dataset['validation']['tokens']
val_tags = dataset['validation']['ner_tags']
val_pos = dataset['validation']['pos_tags']

test_tokens = dataset['test']['tokens']
test_tags = dataset['test']['ner_tags']
test_pos = dataset['test']['pos_tags']

# Print sample sizes
print(f"Training samples: {len(train_tokens)}")
print(f"Validation samples: {len(val_tokens)}")
print(f"Test samples: {len(test_tokens)}")

# Print first example from each split
print("\nFirst training example:")
print("Tokens:", train_tokens[0])
print("Tags:", train_tags[0])
print("POS:", train_pos[0])

print("\nFirst validation example:")
print("Tokens:", val_tokens[0])
print("Tags:", val_tags[0])
print("POS:", val_pos[0])

print("\nFirst test example:")
print("Tokens:", test_tokens[0])
print("Tags:", test_tags[0])
print("POS:", test_pos[0])

Training samples: 2000
Validation samples: 150
Test samples: 250

First training example:
Tokens: ['MRR', ',', 'mortality', 'rate', 'ratio', ';', 'TBI', ',', 'traumatic', 'brain', 'injury', '.']
Tags: ['B-AC', 'O', 'B-LF', 'I-LF', 'I-LF', 'O', 'B-AC', 'O', 'B-LF', 'I-LF', 'I-LF', 'O']
POS: ['NOUN', 'PUNCT', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'NOUN', 'PUNCT']

First validation example:
Tokens: ['Multi', '-', 'locus', 'sequence', 'typing', '(', 'MLST', ')', 'was', 'performed', 'by', 'PCR', '-', 'sequencing', 'of', 'a', 'set', 'of', 'pneumococcal', 'housekeeping', 'genes', ',', 'including', 'ardE', ',', 'gdh', ',', 'gki', ',', 'recP', ',', 'spi', ',', 'xpt', ',', 'and', 'ddl', '[', '17', ']', '.']
Tags: ['B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'O', 'B-AC', 'O', 'O', 'O', 'O', 'B-AC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
POS: ['ADJ', 'PUNCT', 'ADJ', 'NO

# Experiment 1: Varying Vectorization Methods (3 Systems)
**Systems:**
1. **System 1**: CRF with Basic Features (word, prefix, suffix)
2. **System 2**: CRF with Advanced Features (POS tags, n-grams)
3. **System 3**: BERT Embeddings + FFNN

# System 1 : CRF with Basic Features (word, prefix, suffix)

In [23]:
# Variation 1: Basic Features
def basic_featurizer_v1(sent):
    return [{'word': word.lower(), 'suffix3': word[-3:].lower()} for word in sent]

# Variation 2: Extended Features
def basic_featurizer_v2(sent):
    return [{
        'word': word.lower(),
        'prefix3': word[:3].lower(),
        'suffix3': word[-3:].lower(),
        'is_cap': word[0].isupper()
    } for word in sent]

# Initialize model
sys1_crf = CRF(
    algorithm='lbfgs',
    c1=0.5,
    c2=0.01,
    max_iterations=200,
    linesearch='StrongBacktracking',
    all_possible_transitions=True
)

def run_crf_pipeline(featurizer_func, dataset, crf_model, variation_name="Unnamed Variation"):
    print(f"\n=== Running CRF Pipeline: {variation_name} ===\n")

    # Prepare features
    X_train = [featurizer_func(s) for s in dataset['train']['tokens']]
    X_val = [featurizer_func(s) for s in dataset['validation']['tokens']]
    X_test = [featurizer_func(s) for s in dataset['test']['tokens']]

    # Prepare labels
    y_train = dataset['train']['ner_tags']
    y_val = dataset['validation']['ner_tags']
    y_test = dataset['test']['ner_tags']

    # Verify dataset sizes
    assert len(X_train) == len(y_train), "Train feature/label mismatch"
    assert len(X_val) == len(y_val), "Validation feature/label mismatch"
    assert len(X_test) == len(y_test), "Test feature/label mismatch"

    # Train model
    crf_model.fit(X_train, y_train)

    # Validate on validation set
    y_val_pred = crf_model.predict(X_val)
    print("=== Validation Set Report ===")
    print(classification_report(y_val, y_val_pred))

    # Final test on test set
    y_test_pred = crf_model.predict(X_test)
    print("\n=== Test Set Report ===")
    print(classification_report(y_test, y_test_pred))

# Run for both feature variations
run_crf_pipeline(basic_featurizer_v1, dataset, sys1_crf, variation_name="System 1: Basic Featurizer v1")
run_crf_pipeline(basic_featurizer_v2, dataset, sys1_crf, variation_name="System 1: Extended Featurizer v2")


=== Running CRF Pipeline: System 1: Basic Featurizer v1 ===

=== Validation Set Report ===
              precision    recall  f1-score   support

          AC       0.81      0.47      0.60       508
          LF       0.50      0.38      0.43       306

   micro avg       0.67      0.44      0.53       814
   macro avg       0.66      0.43      0.52       814
weighted avg       0.69      0.44      0.54       814


=== Test Set Report ===
              precision    recall  f1-score   support

          AC       0.80      0.44      0.57       797
          LF       0.55      0.43      0.48       482

   micro avg       0.69      0.44      0.53      1279
   macro avg       0.68      0.43      0.52      1279
weighted avg       0.71      0.44      0.54      1279


=== Running CRF Pipeline: System 1: Extended Featurizer v2 ===

=== Validation Set Report ===
              precision    recall  f1-score   support

          AC       0.76      0.79      0.78       508
          LF       0.51  

# System 2 : CRF with Advanced Features (POS tags, n-grams)

In [22]:
import nltk
from nltk.util import ngrams
from nltk.stem import PorterStemmer

# Initialize stemmer
stemmer = PorterStemmer()

def advanced_featurizer(sent):
    # Create n-grams
    bigrams = list(ngrams(sent, 2))
    trigrams = list(ngrams(sent, 3))

    features = []
    for i, word in enumerate(sent):
        # Get surrounding words
        prev_word = sent[i-1].lower() if i > 0 else '<START>'
        next_word = sent[i+1].lower() if i < len(sent)-1 else '<END>'

        # Get n-grams
        curr_bigram = bigrams[i] if i < len(bigrams) else '<END>'
        curr_trigram = trigrams[i] if i < len(trigrams) else '<END>'

        features.append({
            # Word features
            'word': word.lower(),
            'stem': stemmer.stem(word),
            'length': len(word),

            # N-gram features
            'bigram': f"{curr_bigram[0].lower()}_{curr_bigram[1].lower()}",
            'trigram': f"{curr_trigram[0].lower()}_{curr_trigram[1].lower()}_{curr_trigram[2].lower()}",

            # Context features
            'prev_word': prev_word,
            'next_word': next_word,
            'prev_next': f"{prev_word}_{next_word}",

            # Orthographic features
            'is_cap': word[0].isupper(),
            'is_num': word.isdigit(),
            'has_hyphen': '-' in word,
            'prefix3': word[:3].lower(),
            'suffix3': word[-3:].lower()
        })
    return features

# Initialize model
sys2_crf = CRF(
    algorithm='lbfgs',
    c1=0.5,
    c2=0.01,
    max_iterations=200,
    linesearch='StrongBacktracking',
    all_possible_transitions=True
)

# Prepare features
X_train_sys2 = [advanced_featurizer(s) for s in dataset['train']['tokens']]
X_val_sys2 = [advanced_featurizer(s) for s in dataset['validation']['tokens']]
X_test_sys2 = [advanced_featurizer(s) for s in dataset['test']['tokens']]

# Prepare labels
y_train_sys2 = dataset['train']['ner_tags']
y_val_sys2 = dataset['validation']['ner_tags']
y_test_sys2 = dataset['test']['ner_tags']

# Verify dataset sizes
assert len(X_train_sys2) == len(y_train_sys2), "Train feature/label mismatch"
assert len(X_val_sys2) == len(y_val_sys2), "Validation feature/label mismatch"
assert len(X_test_sys2) == len(y_test_sys2), "Test feature/label mismatch"

# Train model
sys2_crf.fit(X_train_sys2, y_train_sys2)

# Validate on validation set
y_val_pred_sys2 = sys2_crf.predict(X_val_sys2)
print("=== Validation Set Report ===")
print(classification_report(y_val_sys2, y_val_pred_sys2))

# Final test on test set
y_test_pred_sys2 = sys2_crf.predict(X_test_sys2)
print("\n=== Test Set Report ===")
print(classification_report(y_test_sys2, y_test_pred_sys2))

=== Validation Set Report ===
              precision    recall  f1-score   support

          AC       0.83      0.76      0.79       508
          LF       0.67      0.64      0.65       306

   micro avg       0.77      0.72      0.74       814
   macro avg       0.75      0.70      0.72       814
weighted avg       0.77      0.72      0.74       814


=== Test Set Report ===
              precision    recall  f1-score   support

          AC       0.85      0.80      0.82       797
          LF       0.71      0.67      0.69       482

   micro avg       0.79      0.75      0.77      1279
   macro avg       0.78      0.73      0.76      1279
weighted avg       0.79      0.75      0.77      1279



# System 3: BERT + FFNN